In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression

# X: стовпець одиниць (bias), площа, кількість ванних кімнат, кількість спалень
X_raw = np.array([
    [1500, 3, 2],
    [2000, 4, 3],
    [1200, 2, 2],
    [2500, 4, 4],
    [1800, 3, 3]
])
# y: ціна будинку
y = np.array([300000, 500000, 250000, 600000, 400000])

print("--- ПОЧАТКОВІ ДАНІ (БЕЗ СТОВПЦЯ ОДИНИЦЬ) ---")
print(f"Матриця ознак X_raw:\n{X_raw}")
print(f"Вектор цільових значень y: {y}\n")





--- ПОЧАТКОВІ ДАНІ (БЕЗ СТОВПЦЯ ОДИНИЦЬ) ---
Матриця ознак X_raw:
[[1500    3    2]
 [2000    4    3]
 [1200    2    2]
 [2500    4    4]
 [1800    3    3]]
Вектор цільових значень y: [300000 500000 250000 600000 400000]



In [10]:
# Обчислюємо середнє та стандартне відхилення для кожної ознаки (кожного стовпця)
mean = np.mean(X_raw, axis=0)
std = np.std(X_raw, axis=0)

print(f"--- Масштабування ---")
print(f"Середнє значення для ознак: {mean}")
print(f"Стандартне відхилення для ознак: {std}\n")

--- Масштабування ---
Середнє значення для ознак: [1800.     3.2    2.8]
Стандартне відхилення для ознак: [442.71887242   0.74833148   0.74833148]



In [11]:
# Застосовуємо стандартизацію
X_scaled = (X_raw - mean) / std

print(f"Масштабована матриця ознак X_scaled:\n{X_scaled}\n")

Масштабована матриця ознак X_scaled:
[[-0.67763093 -0.26726124 -1.06904497]
 [ 0.45175395  1.06904497  0.26726124]
 [-1.35526185 -1.60356745 -1.06904497]
 [ 1.58113883  1.06904497  1.60356745]
 [ 0.         -0.26726124  0.26726124]]



In [12]:
# Додаємо стовпець одиниць (bias) до масштабованих даних
X = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

print(f"Фінальна матриця ознак X (з доданим стовпцем одиниць):\n{X}\n")

Фінальна матриця ознак X (з доданим стовпцем одиниць):
[[ 1.         -0.67763093 -0.26726124 -1.06904497]
 [ 1.          0.45175395  1.06904497  0.26726124]
 [ 1.         -1.35526185 -1.60356745 -1.06904497]
 [ 1.          1.58113883  1.06904497  1.60356745]
 [ 1.          0.         -0.26726124  0.26726124]]



In [13]:
# Гіпотеза
def hypothesis(X, w):
    return np.dot(X, w)

# Функція втрат
def cost_function(X, y, w):
    m = len(y)
    predictions = hypothesis(X, w)
    cost = (1 / (2 * m)) * np.sum(np.square(predictions - y))
    return cost

# Градієнтний спуск (на масштабованих даних)
learning_rate = 0.1
epochs = 200
w = np.zeros(X.shape[1])

print("\n--- ПОЧАТОК НАВЧАННЯ: ГРАДІЄНТНИЙ СПУСК ---")
for i in range(epochs):
    predictions = hypothesis(X, w)
    error = predictions - y
    gradient = (1 / len(y)) * np.dot(X.T, error)
    w = w - learning_rate * gradient
    if i % 40 == 0:
      print(f"Епоха {i}, Вартість: {cost_function(X, y, w):.2f}, Ваги: {w}")

w_gradient_descent = w
print(f"\nПідсумкові параметри, знайдені градієнтним спуском:\nw_gd = {w_gradient_descent}\n")


--- ПОЧАТОК НАВЧАННЯ: ГРАДІЄНТНИЙ СПУСК ---
Епоха 0, Вартість: 72409418367.35, Ваги: [41000.         12649.11064067 11759.49464415 12294.01712797]
Епоха 40, Вартість: 106751476.01, Ваги: [404545.85419461  44201.79271838  40229.3382142   47909.17650635]
Епоха 80, Вартість: 89600855.98, Ваги: [409919.3829093   42465.36786149  39654.85707815  50280.24796643]
Епоха 120, Вартість: 88033919.03, Ваги: [409998.80840822  40656.35573009  39835.16141015  51990.56259989]
Епоха 160, Вартість: 86654105.07, Ваги: [409999.98238722  38847.0630053   40314.67252536  53409.32199845]

Підсумкові параметри, знайдені градієнтним спуском:
w_gd = [409999.99971074  37110.45731059  40890.88394826  54657.97158111]



In [14]:
# Аналітичне рішення (на масштабованих даних)
def normal_equation(X, y):
    print("\n--- РОЗРАХУНОК: АНАЛІТИЧНЕ РІШЕННЯ ---")
    try:
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        return w
    except np.linalg.LinAlgError:
        return "Матриця X.T * X є сингулярною (не має оберненої)"

w_analytical = normal_equation(X, y)
print(f"\nПідсумкові параметри, знайдені аналітично:\nw_analytical = {w_analytical}\n")


--- РОЗРАХУНОК: АНАЛІТИЧНЕ РІШЕННЯ ---

Підсумкові параметри, знайдені аналітично:
w_analytical = [410000.         -27669.92952647  65479.00426854  98218.50640281]



In [15]:
# Перевірка за допомогою scikit-learn
print("\n--- ПЕРЕВІРКА ЗА ДОПОМОГОЮ SCIKIT-LEARN ---")
model = LinearRegression()
model.fit(X_raw, y)

print(f"\nПерехоплення (intercept) з scikit-learn: {model.intercept_}")
print(f"Коефіцієнти (ваги) з scikit-learn: {model.coef_}")


--- ПЕРЕВІРКА ЗА ДОПОМОГОЮ SCIKIT-LEARN ---

Перехоплення (intercept) з scikit-learn: -125000.00000000768
Коефіцієнти (ваги) з scikit-learn: [-6.2500e+01  8.7500e+04  1.3125e+05]


In [16]:
# Порівняння результатів
print("\n--- ПОРІВНЯННЯ ПРОГНОЗІВ для будинку (площа=1600, ванні=3, спальні=2) ---")
sample_house_raw = np.array([1600, 3, 2])

sample_house_scaled = (sample_house_raw - mean) / std
sample_house = np.insert(sample_house_scaled, 0, 1)

print(f"\nПочаткові дані будинку: {sample_house_raw}")
print(f"Масштабовані дані будинку (з одиницею): {sample_house}")


--- ПОРІВНЯННЯ ПРОГНОЗІВ для будинку (площа=1600, ванні=3, спальні=2) ---

Початкові дані будинку: [1600    3    2]
Масштабовані дані будинку (з одиницею): [ 1.         -0.45175395 -0.26726124 -1.06904497]


In [17]:
# Прогноз з градієнтного спуску
prediction_gd = hypothesis(sample_house, w_gradient_descent)
print(f"\nПрогноз (градієнтний спуск): {prediction_gd:.2f}")


Прогноз (градієнтний спуск): 323874.83


In [18]:
# Прогноз з аналітичного рішення
prediction_analytical = hypothesis(sample_house, w_analytical)
print(f"Прогноз (аналітичне рішення):  {prediction_analytical:.2f}")

Прогноз (аналітичне рішення):  300000.00


In [19]:
# Прогноз з scikit-learn (використовуємо початкові, немасштабовані дані)
prediction_sklearn = model.predict([sample_house_raw])[0]
print(f"Прогноз (scikit-learn):        {prediction_sklearn:.2f}")

Прогноз (scikit-learn):        300000.00
